In [1]:
import pyodbc

# Connection settings
server = 'User\SQLEXPRESS'  # Replace with your server name
database = 'gravity_books'   # Database name

connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    "Trusted_Connection=yes;"
)

try:
    # Establish connection
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    print("Connection successful!")

except pyodbc.Error as e:
    print(f"Error: {e}")

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()


Connection successful!


''' import pyodbc

# SQL Server Connection Details
server = 'User\SQLEXPRESS'  # Change to your actual SQL Server name
database = 'gravity_books'  # Connect to 'master' first to create the new database
driver = '{ODBC Driver 17 for SQL Server}'  # Ensure this driver is installed

# Use Windows Authentication
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes')
cursor = conn.cursor()

# Create Database (if not exists)
try:
    cursor.execute("IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = 'gravity_books') CREATE DATABASE gravity_books;")
    print("Database 'gravity_books' created or already exists.")
    cursor.commit()
except Exception as e:
    print("Error creating database:", e)

# Switch to 'gravity_books' database
conn.close()
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE=gravity_books;Trusted_Connection=yes')
cursor = conn.cursor()

# SQL Script for Table Creation
sql_script = """
CREATE TABLE author (
    author_id INT PRIMARY KEY,
    author_name VARCHAR(400)
);

CREATE TABLE publisher (
    publisher_id INT PRIMARY KEY,
    publisher_name VARCHAR(400)
);

CREATE TABLE book_language (
    language_id INT PRIMARY KEY,
    language_code VARCHAR(8),
    language_name VARCHAR(50)
);

CREATE TABLE book (
    book_id INT PRIMARY KEY,
    title VARCHAR(400),
    isbn13 VARCHAR(13),
    language_id INT,
    num_pages INT,
    publication_date DATE,
    publisher_id INT,
    FOREIGN KEY (language_id) REFERENCES book_language (language_id),
    FOREIGN KEY (publisher_id) REFERENCES publisher (publisher_id)
);

CREATE TABLE book_author (
    book_id INT,
    author_id INT,
    PRIMARY KEY (book_id, author_id),
    FOREIGN KEY (book_id) REFERENCES book (book_id),
    FOREIGN KEY (author_id) REFERENCES author (author_id)
);

CREATE TABLE address_status (
    status_id INT PRIMARY KEY,
    address_status VARCHAR(30)
);

CREATE TABLE country (
    country_id INT PRIMARY KEY,
    country_name VARCHAR(200)
);

CREATE TABLE address (
    address_id INT PRIMARY KEY,
    street_number VARCHAR(10),
    street_name VARCHAR(200),
    city VARCHAR(100),
    country_id INT,
    FOREIGN KEY (country_id) REFERENCES country (country_id)
);

CREATE TABLE customer (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(200),
    last_name VARCHAR(200),
    email VARCHAR(350)
);

CREATE TABLE customer_address (
    customer_id INT,
    address_id INT,
    status_id INT,
    PRIMARY KEY (customer_id, address_id),
    FOREIGN KEY (customer_id) REFERENCES customer (customer_id),
    FOREIGN KEY (address_id) REFERENCES address (address_id)
);

CREATE TABLE shipping_method (
    method_id INT PRIMARY KEY,
    method_name VARCHAR(100),
    cost DECIMAL(6, 2)
);

CREATE TABLE cust_order (
    order_id INT IDENTITY PRIMARY KEY,
    order_date DATETIME,
    customer_id INT,
    shipping_method_id INT,
    dest_address_id INT,
    FOREIGN KEY (customer_id) REFERENCES customer (customer_id),
    FOREIGN KEY (shipping_method_id) REFERENCES shipping_method (method_id),
    FOREIGN KEY (dest_address_id) REFERENCES address (address_id)
);

CREATE TABLE order_status (
    status_id INT PRIMARY KEY,
    status_value VARCHAR(20)
);

CREATE TABLE order_line (
    line_id INT IDENTITY PRIMARY KEY,
    order_id INT,
    book_id INT,
    price DECIMAL(5, 2),
    FOREIGN KEY (order_id) REFERENCES cust_order (order_id),
    FOREIGN KEY (book_id) REFERENCES book (book_id)
);

CREATE TABLE order_history (
    history_id INT IDENTITY PRIMARY KEY,
    order_id INT,
    status_id INT,
    status_date DATETIME,
    FOREIGN KEY (order_id) REFERENCES cust_order (order_id),
    FOREIGN KEY (status_id) REFERENCES order_status (status_id)
);
"""

# Execute SQL Statements
try:
    for statement in sql_script.split(";"):
        if statement.strip():
            cursor.execute(statement)
            print("Executed:", statement.strip()[:50])  # Show first 50 chars of query
    conn.commit()
    print("Tables created successfully.")
except Exception as e:
    print("Error executing SQL script:", e)

# Close connection
cursor.close()
conn.close()
'''

In [4]:
import pyodbc

# SQL Server Connection Details
server = 'User\SQLEXPRESS'  # Change to your actual SQL Server name
database = 'gravity_books_dwh'  # Data warehouse database
driver = '{ODBC Driver 17 for SQL Server}'  # Ensure this driver is installed

# Use Windows Authentication
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE=master;Trusted_Connection=yes')
cursor = conn.cursor()

# Create Database (if not exists)
try:
    cursor.execute("SELECT name FROM sys.databases WHERE name = 'gravity_books_dwh'")
    db_exists = cursor.fetchone()

    if not db_exists:
        cursor.execute("CREATE DATABASE gravity_books_dwh;")
        print("Database 'gravity_books_dwh' created.")
    else:
        print("Database 'gravity_books_dwh' already exists.")

    cursor.commit()
except Exception as e:
    print("Error creating database:", e)

# Switch to 'gravity_books_dwh' database
conn.close()
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE=gravity_books_dwh;Trusted_Connection=yes')
cursor = conn.cursor()

# List of table creation queries with proper IF NOT EXISTS checks
star_schema_queries = [
    """
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'DimOrder')
    BEGIN
        CREATE TABLE DimOrder (
            method_id_SK INT IDENTITY PRIMARY KEY,
            method_id_bk INT,
            method_name VARCHAR(100),
            source_system_code VARCHAR(50),
            start_date DATE,
            end_date DATE,
            is_current BIT
        )
    END
    """,
    """
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'DimBook')
    BEGIN
        CREATE TABLE DimBook (
            book_id_SK INT IDENTITY PRIMARY KEY,
            book_id_BK INT,
            language_id_BK INT,
            author_id_BK INT,
            publisher_id_BK INT,
            isbn13 VARCHAR(13),
            publication_date DATE,
            num_pages INT,
            language_name VARCHAR(50),
            publisher_name VARCHAR(200),
            author_name VARCHAR(200),
            source_system_code VARCHAR(50),
            start_date DATE,
            end_date DATE,
            is_current BIT
        )
    END
    """,
    """
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'DimCustomer')
    BEGIN
        CREATE TABLE DimCustomer (
            customer_id_SK INT IDENTITY PRIMARY KEY,
            customer_id_BK INT,
            address_id_BK INT,
            country_id_BK INT,
            status_id_BK INT,
            first_name VARCHAR(100),
            last_name VARCHAR(100),
            email VARCHAR(200),
            country_name VARCHAR(100),
            street_name VARCHAR(200),
            city VARCHAR(100),
            address_status VARCHAR(50),
            source_system_code VARCHAR(50),
            start_date DATE,
            end_date DATE,
            is_current BIT
        )
    END
    """,
    """
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'DimDate')
    BEGIN
        CREATE TABLE DimDate (
            date_Id_SK INT IDENTITY PRIMARY KEY,
            year INT,
            month INT,
            day INT,
            start_date DATE,
            end_date DATE,
            is_current BIT
        )
    END
    """,
    """
    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'FactOrder')
    BEGIN
        CREATE TABLE FactOrder (
            order_id_SK INT IDENTITY PRIMARY KEY,
            line_id INT,
            order_history_id INT,
            book_id INT,
            customer_id INT,
            shipping_method_id INT,
            order_status_id INT,
            order_status VARCHAR(50),
            price DECIMAL(10,2),
            shipping_cost DECIMAL(10,2),
            source_system_code VARCHAR(50),
            created_at DATETIME,
            FOREIGN KEY (book_id) REFERENCES DimBook(book_id_SK),
            FOREIGN KEY (customer_id) REFERENCES DimCustomer(customer_id_SK),
            FOREIGN KEY (shipping_method_id) REFERENCES DimOrder(method_id_SK)
        )
    END
    """
]

# Execute table creation queries separately
try:
    for query in star_schema_queries:
        cursor.execute(query)
        print(f"Executed: {query.strip().split()[6]} table creation.")  # Show table name
    conn.commit()
    print("Star Schema tables created successfully.")
except Exception as e:
    print("Error executing SQL script:", e)

# Close connection
cursor.close()
conn.close()
print("Database setup completed.")


Database 'gravity_books_dwh' already exists.
Executed: INFORMATION_SCHEMA.TABLES table creation.
Executed: INFORMATION_SCHEMA.TABLES table creation.
Executed: INFORMATION_SCHEMA.TABLES table creation.
Executed: INFORMATION_SCHEMA.TABLES table creation.
Executed: INFORMATION_SCHEMA.TABLES table creation.
Star Schema tables created successfully.
Database setup completed.


In [31]:
import pyodbc

# SQL Server Connection Details
server = 'User\SQLEXPRESS'  # Change this to your actual SQL Server name
database = 'gravity_books_dwh'  # Data warehouse database
driver = '{ODBC Driver 17 for SQL Server}'  # Ensure this driver is installed

# Connect to SQL Server
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE=master;Trusted_Connection=yes')
cursor = conn.cursor()

# Check if database exists; if not, create it
cursor.execute("SELECT name FROM sys.databases WHERE name = ?", (database,))
db_exists = cursor.fetchone()

if not db_exists:
    cursor.execute(f"CREATE DATABASE {database};")
    print(f"Database '{database}' created.")
else:
    print(f"Database '{database}' already exists.")

cursor.commit()
conn.close()

# Connect to the newly created database
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes')
cursor = conn.cursor()

# Create Tables (with OBJECT_ID check)
table_queries = [
    """
    IF OBJECT_ID('dbo.DimOrder', 'U') IS NULL
    CREATE TABLE DimOrder (
        method_id_SK INT IDENTITY PRIMARY KEY,
        method_id_bk INT,
        method_name VARCHAR(100),
        source_system_code VARCHAR(50),
        start_date DATE,
        end_date DATE,
        is_current BIT
    )
    """,
    """
    IF OBJECT_ID('dbo.DimBook', 'U') IS NULL
    CREATE TABLE DimBook (
        book_id_SK INT IDENTITY PRIMARY KEY,
        book_id_bk INT,
        language_id_BK INT,
        author_id_BK INT,
        publisher_id_BK INT,
        isbn13 VARCHAR(13),
        publication_date DATE,
        num_pages INT,
        language_name VARCHAR(50),
        publisher_name VARCHAR(200),
        author_name VARCHAR(200),
        source_system_code VARCHAR(50),
        start_date DATE,
        end_date DATE,
        is_current BIT
    )
    """,
    """
    IF OBJECT_ID('dbo.DimCustomer', 'U') IS NULL
    CREATE TABLE DimCustomer (
        customer_id_SK INT IDENTITY PRIMARY KEY,
        customer_id_bk INT,
        address_id_BK INT,
        country_id_BK INT,
        status_id_BK INT,
        first_name VARCHAR(100),
        last_name VARCHAR(100),
        email VARCHAR(200),
        country_name VARCHAR(100),
        street_name VARCHAR(200),
        city VARCHAR(100),
        address_status VARCHAR(50),
        source_system_code VARCHAR(50),
        start_date DATE,
        end_date DATE,
        is_current BIT
    )
    """,
    """
    IF OBJECT_ID('dbo.DimDate', 'U') IS NULL
    CREATE TABLE DimDate (
        date_id_SK INT IDENTITY PRIMARY KEY,
        year INT,
        month INT,
        day INT,
        start_date DATE,
        end_date DATE,
        is_current BIT
    )
    """,
    """
    IF OBJECT_ID('dbo.FactOrder', 'U') IS NULL
    CREATE TABLE FactOrder (
        order_id_SK INT IDENTITY PRIMARY KEY,
        line_id INT,
        order_id_BK INT,
        order_history_id INT,
        book_id INT,
        customer_id INT,
        shipping_id INT,
        order_status_id INT,
        date_id_SK INT,  -- Changed from order_date (DATE) to date_id_SK (INT)
        order_status VARCHAR(50),
        price DECIMAL(10,2),
        shipping_cost DECIMAL(10,2),
        source_system_code VARCHAR(50),
        created_at DATETIME,
        FOREIGN KEY (book_id) REFERENCES DimBook(book_id_SK),
        FOREIGN KEY (customer_id) REFERENCES DimCustomer(customer_id_SK),
        FOREIGN KEY (shipping_id) REFERENCES DimOrder(method_id_SK),
        FOREIGN KEY (date_id_SK) REFERENCES DimDate(date_id_SK)  -- Fixed the incorrect FK reference
    )
    """
]

# Execute table creation queries
try:
    for query in table_queries:
        cursor.execute(query)
        print(f"Table created (or already exists): {query.strip().split()[5]}")
    conn.commit()
    print("✅ All tables created successfully.")
except Exception as e:
    print("⚠️ Error creating tables:", e)

# Close connection
cursor.close()
conn.close()
print("Database setup completed.")


Database 'gravity_books_dwh' already exists.
Table created (or already exists): CREATE
Table created (or already exists): CREATE
Table created (or already exists): CREATE
Table created (or already exists): CREATE
Table created (or already exists): CREATE
✅ All tables created successfully.
Database setup completed.
